## Lab

Implement a URL fetcher using Beautiful Soup in the callback version. We will implement a similar one using coroutines on wednesday. 

The implimentation will extend the read_response method by parsing for URL's using `bs4` . Start by creating globals:
```
urls_todo = set(['/'])
seen_urls = set(['/'])
```

then:

```
links = self.parse_links()#write this
```
(using self.response)

Then use the set `difference` method  to add new links to `urls_todo` and recursively set up a `Fetcher` instance.

Now update the `seen_urls` and `urls_todo` thus:
```
seen_urls.update(links)
urls_todo.remove(self.url)
if not urls_todo:
    stopped = True
```

In [179]:
import socket
from bs4 import BeautifulSoup
from selectors import DefaultSelector, EVENT_READ, EVENT_WRITE

selector = DefaultSelector()
class Fetcher:
    def __init__(self, host, url, level = 0):
        self.response = b''  # Empty array of bytes.
        self.host = host
        self.url = url
        self.sock = None
        self.level = level
        
    # Method on Fetcher class.
    def fetch(self):
        self.sock = socket.socket()
        self.sock.setblocking(False)
        try:
            self.sock.connect((self.host, 80))
        except BlockingIOError:
            pass

        # Register next callback.
        selector.register(self.sock.fileno(),
                          EVENT_WRITE,
                          self.connected)

    def connected(self, key, mask):
        print('connected to:', self.url, flush=True)
        selector.unregister(key.fd)
        request = 'GET {} HTTP/1.0\r\nHost: {}\r\n\r\n'.format(self.url, self.host)
        self.sock.send(request.encode('ascii'))

        # Register the next callback.
        selector.register(key.fd,
                          EVENT_READ,
                          self.read_response)
        
    def read_response(self, key, mask):
        global stopped
        
        chunk = self.sock.recv(4096)  # USUALLY 4k chunk size, here small
        if chunk:
            #print("read chunk", flush=True)
            self.response += chunk
        else:
            print("all read", flush=True)
            links = self.parse_links()
            selector.unregister(key.fd)  # Done reading.
            print ("Links in:", self.url, " are:",links)         
            
            links.difference_update(seen_urls)
            #print (links)
            if self.level < 1:
                for link in links:
                    urls_todo.add(link)
                    
            if self.level < 1:
                for link in links:
                    urls_todo.add(link)
                    fetcher = Fetcher(self.host, link, self.level+1)
                    fetcher.fetch()

            seen_urls.update(links)
            urls_todo.remove(self.url)
            #print (len(urls_todo), self.url, urls_todo)
            #print (list(selector.get_map()))
            if not urls_todo:
                stopped = True
            
            
    def parse_links(self):
        soup = BeautifulSoup(self.response, "lxml");
        #print (soup)
        result = set([])
        for link in soup.find_all('a'):
            linkurl = link.get('href')
            if linkurl is not None and linkurl.startswith('/'):
                #print (linkurl)
                result.add(linkurl)
        #for x in range(3):
        #    host = 'xkcd.com'
        #    url = '/37'+str(x)+'/'
        #    result.add(url)
        #print (result)
        return result
            
stopped = False

def loop():
    while not stopped:
        events = selector.select()
        for event_key, event_mask in events:
            callback = event_key.data
            callback(event_key, event_mask)

In [180]:
urls_todo = set(['/'])
seen_urls = set(['/'])

fetcher = Fetcher('xkcd.com','/')
fetcher.fetch()
loop()

connected to: /
all read
Links in: /  are: {'/about', '/', '/atom.xml', '/license.html', '/1660/', '//c.xkcd.com/random/comic/', '/archive', '/1/', '/rss.xml'}
connected to: /atom.xml
connected to: /about
connected to: /license.html
connected to: /archive
connected to: //c.xkcd.com/random/comic/
connected to: /1660/
connected to: /1/
connected to: /rss.xml
all read
Links in: /about  are: set()
all read
Links in: //c.xkcd.com/random/comic/  are: set()
all read
Links in: /rss.xml  are: set()
all read
Links in: /1660/  are: {'/about', '/', '/atom.xml', '/license.html', '//c.xkcd.com/random/comic/', '/archive', '/1659/', '/1/', '/1661/', '/rss.xml'}
all read
Links in: /atom.xml  are: set()
all read
Links in: /license.html  are: set()
all read
Links in: /archive  are: set()
all read
Links in: /1/  are: {'/about', '/', '/atom.xml', '/license.html', '/2/', '//c.xkcd.com/random/comic/', '/archive', '/1/', '/rss.xml'}
